In [1]:
import warnings
import numpy as np
import pandas as pd

from utils import cage, sm_present, mh_stress, eq_5d

warnings.filterwarnings('ignore')

In [2]:
try:
    df = pd.read_csv('./dataset/HN07-19_ALL_fev1fvc.csv')
    df = df[np.logical_and(df['age'] > 40, df['HE_COPD'] == 3)].drop('HE_COPD', axis=1)
except:
    pd.read_excel('./dataset/HN07-19_ALL_fev1fvc.xlsx').to_csv('./dataset/HN07-19_ALL_fev1fvc.csv', index=False)
    df = pd.read_csv('./dataset/HN07-19_ALL_fev1fvc.csv')
    df = df[np.logical_and(df['age'] > 40, df['HE_COPD'] == 3)].drop('HE_COPD', axis=1)

In [3]:
df.drop(['id', 'year', 'psu'], axis=1, inplace=True)
df.replace('.', np.nan, inplace=True)

In [4]:
(df.isna().sum().sort_values(ascending=False) / df.shape[0] * 100)[:5]

HE_PFThs     80.146351
HE_cough2    77.065280
HE_sput2     71.403813
BH9_11        8.184094
HE_DM         7.067206
dtype: float64

In [5]:
df.drop(['HE_PFThs', 'HE_cough2', 'HE_sput2'], axis=1, inplace=True)
df.drop(['kstrata', 'wt_itvex'], axis=1, inplace=True)

In [6]:
for col in df.columns:
    try:
        df[col] = df[col].astype(float)
    except:
        print(col)

In [7]:
df['cage'] = df.apply(
    lambda row: cage(row['age']), axis=1)
df['sm_present'] = df.apply(
    lambda row: sm_present(row['BS1_1'], row['BS3_1']), axis=1)
df['mh_stress'] = df.apply(
    lambda row: mh_stress(row['BP1']), axis=1)
df['eq_5d'] = df.apply(
    lambda row: eq_5d(row['LQ_1EQL'], row['LQ_2EQL'], row['LQ_3EQL'], row['LQ_4EQL'], row['LQ_5EQL']), axis=1)

df['BE3_31'] = df.pop('BE3_31')
df['BE5_1'] = df.pop('BE5_1')

In [8]:
df.head(2)

,BS3_1,HE_FEV1FVC,age,sex,occp,edu,EC1_1,cfam,marri_1,BH9_11,...,BD7_4,BD7_5,BP6_10,BP6_31,cage,sm_present,mh_stress,eq_5d,BE3_31,BE5_1
1,2.0,0.699366,74.0,1.0,7.0,1.0,2.0,2.0,1.0,2.0,...,1.0,2.0,2.0,8.0,7,1,1.0,0.72,8.0,1.0
2,2.0,0.699220,69.0,1.0,4.0,3.0,1.0,2.0,1.0,1.0,...,3.0,2.0,2.0,8.0,6,1,0.0,1.00,8.0,1.0


In [9]:
for col in df.columns:
    print(f'<<{col}>>')
    print(df[col].unique())

<<BS3_1>>
[ 2.  8.  1. nan  3.  9.]
<<HE_FEV1FVC>>
[0.69936613 0.69921973 0.69863014 ... 0.32244898 0.31692308 0.29109589]
<<age>>
[74. 69. 80. 49. 77. 43. 61. 72. 42. 70. 76. 57. 58. 79. 54. 59. 53. 60.
 73. 71. 63. 64. 67. 55. 68. 46. 66. 56. 48. 75. 44. 65. 52. 41. 45. 62.
 78. 51. 47. 50.]
<<sex>>
[1. 2.]
<<occp>>
[ 7.  4.  3.  6.  1.  5. nan  2.]
<<edu>>
[ 1.  3.  4.  2. nan]
<<EC1_1>>
[ 2.  1.  9. nan]
<<cfam>>
[ 2.  1.  3.  4.  5.  6. nan  9.]
<<marri_1>>
[ 1.  2. nan  9.]
<<BH9_11>>
[ 2.  1. nan  9.]
<<HE_DM>>
[ 1.  2.  3. nan  8.  0.]
<<DC6_dg>>
[ 8.  1. nan  9.  0.]
<<DF2_dg>>
[ 8.  0.  1. nan  9.]
<<HE_HPdg>>
[0. 1.]
<<BP1>>
[ 2.  3.  4.  1. nan  9.]
<<BP5>>
[ 2.  1. nan  9.  8.]
<<D_1_1>>
[ 4.  3.  2.  5.  1. nan  9.]
<<LQ_1EQL>>
[ 2.  1.  3. nan  9.]
<<LQ_2EQL>>
[ 2.  1.  3. nan  9.]
<<LQ_3EQL>>
[ 2.  1.  3. nan  9.]
<<LQ_4EQL>>
[ 2.  1.  3.  9. nan]
<<LQ_5EQL>>
[ 1.  2.  3. nan  9.]
<<Total_slp_wk>>
[8.000e+00 9.900e+01 6.000e+00 7.000e+00 3.000e+00 4.000e+00 5.000e+00
 9

In [10]:
# 1차년도
NUMERIC = ['HE_FEV1FVC', 'age', 'Total_slp_wk']
CATEGORICAL = ['sex', 'occp', 'EC1_1', 'cfam', 'marri_1', 'BH9_11', 'HE_DM', 'DC6_dg', 'DF2_dg', 'HE_HPdg', 'BP5']
ONE_HOT = ['BS3_1', 'edu', 'BP1', 'D_1_1', 'LQ_1EQL', 'LQ_2EQL', 'LQ_3EQL', 'LQ_4EQL', 'LQ_5EQL', 'BO1_1', 'BO2_1']
# 2차년도
NUMERIC2 = ['DI1_ag', 'DE1_ag', 'LQ1_mn', 'BS6_2_1', 'BS6_3']
CATEGORICAL2 = ['DI1_pt', 'DE1_pt', 'DE1_3', 'BH1', 'BH2_61', 'LQ4_00', 'LQ4_05', 'LQ1_sb', 'MO1_wk', 'HE_cough1', 'HE_sput1', 'BD1', 'BD7_5', 'BP6_10', 'BP6_31']
ONE_HOT2 = ['DI1_2', 'EC_pedu_1', 'EC_pedu_2', 'BS5_1', 'BD7_4']
# 생성변수
NUMERIC3 = ['eq_5d']
CATEGORICAL3 = ['sm_present', 'mh_stress']
ONE_HOT3 = ['cage']
# Targets
TARGETS = ['BE3_31', 'BE5_1']

In [11]:
df1 = df[NUMERIC + CATEGORICAL + ONE_HOT]
df2 = df[NUMERIC2 + CATEGORICAL2 + ONE_HOT2]
df3 = df[NUMERIC3 + CATEGORICAL3 + ONE_HOT3]
dfT = df[TARGETS]

In [12]:
## 1차년도 변수 수정
# BS3_1(categorical_onehot)
df1['BS3_1'].replace({
    1:'매일피움',
    2:'가끔피움',
    3:'과거에만피움',
    8:'비해당(비흡연)',
    9:np.nan,
}, inplace=True)

# sex(categorical)
df1['sex'].replace(2, 0, inplace=True)

# occp(categorical)
df1.loc[df1['occp'] < 7, 'occp'] = 1
df1['occp'].replace(7, 0, inplace=True)

# edu(categorical_onehot)
df1['edu'].replace({
    1:'초졸이하',
    2:'중졸',
    3:'고졸',
    4:'대졸이상',
}, inplace=True)

# EC1_1(categorical)
df1['EC1_1'].replace({2: 0, 9: np.nan}, inplace=True)

#cfam(categorical_label)
df1['cfam'].replace(9, np.nan, inplace=True)

# marri_1(categorical)
df1['marri_1'].replace({2: 0, 9: np.nan}, inplace=True)

# BH9_11(categorical)
df1['BH9_11'].replace({2: 0, 9: np.nan}, inplace=True)

# HE_DM(categorical)_8:의사진단받지않음
df1['HE_DM'].replace([8, 9], np.nan, inplace=True)

# DC6_dg(categorical)_8:의사진단받지않음
df1['DC6_dg'].replace([8, 9], np.nan, inplace=True)

# DF2_dg(categorical)_8:의사진단받지않음
df1['DF2_dg'].replace([8, 9], np.nan, inplace=True)

# HE_HPdg(categorical)
df1['HE_HPdg'].replace([8, 9], np.nan, inplace=True)

# BP1(categorical_onehot)
df1['BP1'].replace({
    1: '대단히많이', 
    2: '많이', 
    3: '조금', 
    4: '거의조금',
    9: np.nan,
}, inplace=True)

# BP5(categorical)
df1['BP5'].replace({2: 0, 8: np.nan, 9: np.nan}, inplace=True)

# D_1_1(categorical_onehot)
df1['D_1_1'].replace({
    1: '매우좋음', 
    2: '좋음', 
    3: '보통', 
    4: '나쁨', 
    5: '매우나쁨', 
    9: np.nan
}, inplace=True)

# LQ_1EQL~5EQL(categorical_onehot)
df1['LQ_1EQL'].replace({
    1:'지장없음',
    2:'지장있음',
    3:'불가능',
    9:np.nan,
}, inplace=True)
df1['LQ_2EQL'].replace({
    1:'지장없음',
    2:'지장있음',
    3:'불가능',
    9:np.nan,
}, inplace=True)
df1['LQ_3EQL'].replace({
    1:'지장없음',
    2:'지장있음',
    3:'불가능',
    9:np.nan,
}, inplace=True)
df1['LQ_4EQL'].replace({
    1:'지장없음',
    2:'지장있음',
    3:'불가능',
    9:np.nan,
}, inplace=True)
df1['LQ_5EQL'].replace({
    1:'지장없음',
    2:'지장있음',
    3:'불가능',
    9:np.nan,
}, inplace=True)

# Total_slp_wk(numeric)
df1['Total_slp_wk'].replace([99, 9999], np.nan, inplace=True)

# BO1_1(categorical_onehot)
df1['BO1_1'].replace({
    1: '변화없음', 
    2: '체중감소', 
    3: '체중증가', 
    9: np.nan
}, inplace=True)

# BO2_1(categorical_onehot)
df1['BO2_1'].replace({
    1: '감소노력', 
    2: '유지노력', 
    3: '증가노력',
    4:'노력해본적없음',
    9: np.nan,
}, inplace=True)

In [13]:
## 2차년도 변수 수정
# DI1_ag(numeric)
df2['DI1_ag'].replace([888, 999], np.nan, inplace=True)

# DE1_ag(numeric)
df2['DE1_ag'].replace([888, 999], np.nan, inplace=True)

# BS6_2_1(numeric)
df2['BS6_2_1'].replace({
    88: 0,
    99: np.nan
}, inplace=True)

# BS6_3(numeric)
df2['BS6_3'].replace({
    888: 0,
    999: np.nan
}, inplace=True)

# DI1_pt(categorical)
df2['DI1_pt'].replace([8, 9], np.nan, inplace=True)

# DE1_pt(categorical)
df2['DE1_pt'].replace([8, 9], np.nan, inplace=True)

# DE1_3(categorical)
df2['DE1_3'].replace([8, 9], np.nan, inplace=True)

# BH1(categorical)
df2['BH1'].replace([8, 9], np.nan, inplace=True)

# BH2_61(categorical)
df2['BH2_61'].replace(9, np.nan, inplace=True)

# LQ4_00(categorical)
df2['LQ4_00'].replace([8, 9], np.nan, inplace=True)

# LQ4_05(categorical)
df2['LQ4_05'].replace([8, 9], np.nan, inplace=True)

# LQ1_sb(categorical)
df2['LQ1_sb'].replace(2, 0, inplace=True)
df2['LQ1_sb'].replace([8, 9], np.nan, inplace=True)

# LQ1_mn(numeric)
df2.loc[np.logical_and(df2['LQ1_sb'] == 0, df2['LQ1_mn']==88), 'LQ1_mn'] = 0
df2['LQ1_mn'].replace([88, 99], np.nan, inplace=True)

# MO1_wk(categorical)
df2['MO1_wk'].replace({
    2:0,
    9:np.nan
}, inplace=True)

# HE_cough1(categorical)
df2['HE_cough1'].replace([2, 8, 9], np.nan, inplace=True)

# HE_sput1(categorical)
df2['HE_sput1'].replace([2, 8, 9], np.nan, inplace=True)

# BD1(categorical)
df2['BD1'].replace({
    2:0,
    8:np.nan, 9:np.nan,
}, inplace=True)

# BD7_5(categorical)
df2['BD7_5'].replace({
    2:0,
    8:np.nan, 9:np.nan,
}, inplace=True)

# BP6_10(categorical)
df2['BP6_10'].replace({
    2:0,
    8:np.nan, 9:np.nan,
}, inplace=True)

# BP6_31(categorical)
df2['BP6_31'].replace({
    2:0,
    8:np.nan, 9:np.nan,
}, inplace=True)

# DI1_2(categorical_onehot)
df2['DI1_2'].replace({
    1:'매일복용',
    2:'20일이상',
    3:'15일이상',
    4:'15일미만',
    5:'미복용',
    8:np.nan, 9:np.nan,
}, inplace=True)

# EC_pedu_1 ~ 2(categorical_onehot)
df2['EC_pedu_1'].replace({
    1:'무학',
    2:'서당/한학',
    3:'초졸이하',
    4:'중졸이하',
    5:'고졸이하',
    6:'전문대졸이하',
    7:'대졸이하',
    8:'대학원이상',
    9:np.nan, 88:np.nan, 99:np.nan,
}, inplace=True)
df2['EC_pedu_2'].replace({
    1:'무학',
    2:'서당/한학',
    3:'초졸이하',
    4:'중졸이하',
    5:'고졸이하',
    6:'전문대졸이하',
    7:'대졸이하',
    8:'대학원이상',
    9:np.nan, 88:np.nan, 99:np.nan,
}, inplace=True)

# BS5_1(categorical_onehot)
df2['BS5_1'].replace({
    1:'1개월내금연',
    2:'6개월내금연',
    3:'언젠가금연',
    4:'금연안함',
    8:'비흡연자',
    9:np.nan,
}, inplace=True)

# BD7_4(categorical_onehot)
df2['BD7_4'].replace({
    1:'없었음',
    2:'과거에만1년동안없었음',
    3:'1년동안있었음',
    8:np.nan, 9:np.nan,
}, inplace=True)

In [14]:
for col in dfT.columns:
    print(col)
    print(dfT[col].unique())

BE3_31
[ 8.  1.  5.  2.  3.  4.  6.  7.  9. nan 99.]
BE5_1
[ 1.  3.  6.  4.  5.  2. nan  9.]


In [15]:
# BE3_31(Target1)
dfT['BE3_31'].replace([9, 99], np.nan, inplace=True)
dfT['BE3_31'] = dfT['BE3_31'] - 1

# BE5_1(Target2)
dfT['BE5_1'].replace(9, np.nan, inplace=True)
dfT['BE5_1'] = dfT['BE5_1'] - 1

In [16]:
# 1차년도 변수 결측치
(df1.isna().sum().sort_values(ascending=False) / df1.shape[0] * 100)[:5]

DC6_dg    44.482958
DF2_dg    39.302908
BP5       25.996534
HE_DM     14.500289
BH9_11    10.552667
dtype: float64

In [17]:
df1.drop(['DC6_dg', 'DF2_dg'], axis=1, inplace=True)

In [18]:
# 2차년도 변수 결측치
(df2.isna().sum().sort_values(ascending=False) / df2.shape[0] * 100)[:10]

DE1_pt       84.883497
DE1_ag       84.864240
DE1_3        84.806470
LQ4_05       84.556133
DI1_ag       59.888311
DI1_pt       59.676488
DI1_2        59.618718
BP6_31       44.271134
EC_pedu_1    27.748893
BP6_10       26.092817
dtype: float64

In [19]:
df2.drop(['DE1_pt', 'DE1_ag', 'DE1_3', 'LQ4_05', 'DI1_ag', 'DI1_pt', 'DI1_2', 'BP6_31'], axis=1, inplace=True)

In [20]:
# 생성 변수 결측치
(df3.isna().sum().sort_values(ascending=False) / df3.shape[0] * 100)[:5]

mh_stress     1.675332
eq_5d         0.000000
sm_present    0.000000
cage          0.000000
dtype: float64

In [21]:
# 타겟 변수 결측치
(dfT.isna().sum().sort_values(ascending=False) / dfT.shape[0] * 100)[:5]

BE3_31    3.543231
BE5_1     3.427691
dtype: float64

In [22]:
ff = pd.concat([df1, df2], axis=1)
ff = pd.concat([ff, df3], axis=1)
ff = pd.concat([ff, dfT], axis=1)

In [23]:
ff = ff[ff['Total_slp_wk'] <= 24]
ff.to_csv('./dataset/ff.csv', index=False)